Сначала попробую линейную регрессию

In [104]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
df=pd.read_csv('train_data.csv')

df

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,"8,79",1,"3,86",68323.0,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,"8,66",3,"3,91",28785.0,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
2,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,"8,43",4,"3,24",87099.0,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
3,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,"8,70",5,"4,22",16831.0,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"
4,291457.0,Gloomhaven: Jaws of the Lion,2020.0,1,4,120,14,8392,"8,87",6,"3,55",21609.0,"Action Queue, Campaign / Battle Card Driven, C...","Strategy Games, Thematic Games"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,1410.0,Trouble,1965.0,2,4,45,4,3255,"3,79",20339,"1,05",4962.0,Roll / Spin and Move,Children's Games
15253,16398.0,War,0.0,2,2,30,4,1340,"2,28",20340,1,427.0,NaN,Children's Games
15254,5048.0,Candy Land,1949.0,2,4,30,3,4006,"3,18",20342,"1,08",5788.0,Roll / Spin and Move,Children's Games
15255,5432.0,Chutes and Ladders,-200.0,2,6,30,3,3783,"2,86",20343,"1,02",4400.0,"Dice Rolling, Grid Movement, Race, Roll / Spin...",Children's Games


In [42]:
df.shape

(15257, 14)

In [43]:
df.isna().sum()
#пропуски буду считать как отсутствие информации, то есть 0, только нужно учитывать где 0 как число а где как слово

ID                      10
Name                     0
Year Published           1
Min Players              0
Max Players              0
Play Time                0
Min Age                  0
Users Rated              0
Rating Average           0
BGG Rank                 0
Complexity Average       0
Owned Users             17
Mechanics             1200
Domains               7649
dtype: int64

In [44]:
df.dtypes

ID                    float64
Name                   object
Year Published        float64
Min Players             int64
Max Players             int64
Play Time               int64
Min Age                 int64
Users Rated             int64
Rating Average         object
BGG Rank                int64
Complexity Average     object
Owned Users           float64
Mechanics              object
Domains                object
dtype: object

In [105]:
df['Domains'] = df['Domains'].fillna('0')
df['Mechanics'] = df['Mechanics'].fillna('0')
df['Owned Users'] = df['Owned Users'].fillna(0)

df['Year Published'] = df['Year Published'].fillna(0)
df['ID'] = df['ID'].fillna('0')

In [81]:
df.isna().sum()

ID                    0
Name                  0
Year Published        0
Min Players           0
Max Players           0
Play Time             0
Min Age               0
Users Rated           0
Rating Average        0
BGG Rank              0
Complexity Average    0
Owned Users           0
Mechanics             0
Domains               0
dtype: int64

In [46]:
df['Rating Average'].unique()

array(['8,79', '8,66', '8,43', '8,70', '8,87', '8,49', '8,42', '8,30',
       '8,35', '8,13', '8,14', '8,11', '8,12', '8,17', '8,08', '8,09',
       '7,99', '8,02', '7,94', '8,00', '8,06', '8,15', '7,86', '7,90',
       '8,05', '8,01', '8,10', '7,87', '7,93', '7,83', '8,20', '7,84',
       '7,76', '7,75', '7,95', '7,81', '7,82', '7,91', '7,85', '8,18',
       '8,07', '7,80', '8,04', '8,41', '7,77', '7,88', '7,72', '7,79',
       '7,67', '7,89', '7,73', '7,61', '7,92', '7,57', '8,21', '7,58',
       '7,69', '7,78', '7,68', '7,65', '7,60', '8,80', '7,51', '7,56',
       '7,71', '7,54', '7,55', '7,64', '7,53', '7,52', '7,70', '7,48',
       '7,42', '8,45', '7,47', '8,16', '7,46', '7,59', '7,50', '7,44',
       '7,49', '7,45', '7,41', '7,35', '7,40', '7,63', '8,25', '7,66',
       '7,37', '8,22', '7,43', '7,26', '7,39', '7,30', '7,34', '7,28',
       '7,38', '7,27', '7,23', '7,62', '7,31', '7,36', '7,25', '7,29',
       '7,22', '8,46', '8,78', '7,97', '7,33', '7,32', '7,19', '7,21',
      

In [9]:
df['Rating Average'] = df['Rating Average'].astype(float)
#класс, как я понял все числа записаны с ,

ValueError: could not convert string to float: '8,79'

In [10]:
df.dtypes

ID                     object
Name                   object
Year Published        float64
Min Players             int64
Max Players             int64
Play Time               int64
Min Age                 int64
Users Rated             int64
Rating Average         object
BGG Rank                int64
Complexity Average     object
Owned Users           float64
Mechanics              object
Domains                object
dtype: object

In [106]:
df['Rating Average']=df['Rating Average'].str.replace(',', '.', 15256)
df['Rating Average']=df['Rating Average'].astype(float)
df['Complexity Average']=df['Complexity Average'].str.replace(',','.')
df['Complexity Average']=df['Complexity Average'].astype(float)
df.dtypes

ID                     object
Name                   object
Year Published        float64
Min Players             int64
Max Players             int64
Play Time               int64
Min Age                 int64
Users Rated             int64
Rating Average        float64
BGG Rank                int64
Complexity Average    float64
Owned Users           float64
Mechanics              object
Domains                object
dtype: object

In [48]:
df['Mechanics'].value_counts()

Mechanics
0                                                                                                                                                                   1200
Hand Management                                                                                                                                                      327
Hexagon Grid                                                                                                                                                         299
Dice Rolling                                                                                                                                                         276
Roll / Spin and Move                                                                                                                                                 264
                                                                                                                                                 

In [14]:
df['Domains'].value_counts()

Domains
0                                                 7649
Wargames                                          2236
Strategy Games                                    1103
Family Games                                      1004
Abstract Games                                     653
Children's Games                                   526
Thematic Games                                     485
Party Games                                        305
Family Games, Strategy Games                       269
Customizable Games                                 190
Strategy Games, Thematic Games                     164
Thematic Games, Wargames                           110
Family Games, Party Games                          101
Family Games, Thematic Games                        89
Abstract Games, Family Games                        84
Children's Games, Family Games                      73
Strategy Games, Wargames                            71
Abstract Games, Strategy Games                      26
Cu

In [107]:
df0=df
df1=df
df2=df
df3=df
df4=df
df5=df
df0

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,8.79,1,3.86,68323.0,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,8.66,3,3.91,28785.0,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
2,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,8.43,4,3.24,87099.0,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
3,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,8.70,5,4.22,16831.0,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"
4,291457.0,Gloomhaven: Jaws of the Lion,2020.0,1,4,120,14,8392,8.87,6,3.55,21609.0,"Action Queue, Campaign / Battle Card Driven, C...","Strategy Games, Thematic Games"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,1410.0,Trouble,1965.0,2,4,45,4,3255,3.79,20339,1.05,4962.0,Roll / Spin and Move,Children's Games
15253,16398.0,War,0.0,2,2,30,4,1340,2.28,20340,1.00,427.0,0,Children's Games
15254,5048.0,Candy Land,1949.0,2,4,30,3,4006,3.18,20342,1.08,5788.0,Roll / Spin and Move,Children's Games
15255,5432.0,Chutes and Ladders,-200.0,2,6,30,3,3783,2.86,20343,1.02,4400.0,"Dice Rolling, Grid Movement, Race, Roll / Spin...",Children's Games


Сначала попробую labelEncoder, потом get_dummies и сравню результаты, что лучше то и оставлю

In [108]:
df_gd = df1['Mechanics'].str.get_dummies(sep=', ')
df1 = pd.concat([df1.drop(columns=['Mechanics']), df_gd], axis=1)

df_gd = df1['Domains'].str.get_dummies(sep=', ')
df1 = pd.concat([df1.drop(columns=['Domains']), df_gd], axis=1)
df1

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,...,Zone of Control,0,Abstract Games,Children's Games,Customizable Games,Family Games,Party Games,Strategy Games,Thematic Games,Wargames
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,8.79,1,...,0,0,0,0,0,0,0,1,1,0
1,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,8.66,3,...,0,0,0,0,0,0,0,1,0,0
2,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,8.43,4,...,0,0,0,0,0,0,0,1,0,0
3,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,8.70,5,...,0,0,0,0,0,0,0,1,1,0
4,291457.0,Gloomhaven: Jaws of the Lion,2020.0,1,4,120,14,8392,8.87,6,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,1410.0,Trouble,1965.0,2,4,45,4,3255,3.79,20339,...,0,0,0,1,0,0,0,0,0,0
15253,16398.0,War,0.0,2,2,30,4,1340,2.28,20340,...,0,0,0,1,0,0,0,0,0,0
15254,5048.0,Candy Land,1949.0,2,4,30,3,4006,3.18,20342,...,0,0,0,1,0,0,0,0,0,0
15255,5432.0,Chutes and Ladders,-200.0,2,6,30,3,3783,2.86,20343,...,0,0,0,1,0,0,0,0,0,0


In [109]:
x = df1.drop(['ID', 'Name', 'Rating Average'], axis=1)
y = df1['Rating Average']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [110]:
model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

# print('Веса всех признаков (x1, ..., x16): ', model.coef_)
# print('Свободный коэффицент: ', model.intercept_)

print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
print('R2: ', r2_score(y_test, y_pred))
#Результат очень даже не плох


Mean Absolute Error:  0.4073251350462282
Mean Squared Error:  0.31601817323676074
R2:  0.6528336057477284


In [87]:
df0

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,8.79,1,3.86,68323.0,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,8.66,3,3.91,28785.0,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
2,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,8.43,4,3.24,87099.0,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
3,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,8.70,5,4.22,16831.0,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"
4,291457.0,Gloomhaven: Jaws of the Lion,2020.0,1,4,120,14,8392,8.87,6,3.55,21609.0,"Action Queue, Campaign / Battle Card Driven, C...","Strategy Games, Thematic Games"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,1410.0,Trouble,1965.0,2,4,45,4,3255,3.79,20339,1.05,4962.0,Roll / Spin and Move,Children's Games
15253,16398.0,War,0.0,2,2,30,4,1340,2.28,20340,1.00,427.0,0,Children's Games
15254,5048.0,Candy Land,1949.0,2,4,30,3,4006,3.18,20342,1.08,5788.0,Roll / Spin and Move,Children's Games
15255,5432.0,Chutes and Ladders,-200.0,2,6,30,3,3783,2.86,20343,1.02,4400.0,"Dice Rolling, Grid Movement, Race, Roll / Spin...",Children's Games


In [111]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(df0['Mechanics'])
df0['Mechanics'] = label_encoder.transform(df0['Mechanics'])

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(df0['Domains'])
df0['Domains'] = label_encoder.transform(df0['Domains'])

df0

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,8.79,1,3.86,68323.0,739,34
1,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,8.66,3,3.91,28785.0,4816,33
2,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,8.43,4,3.24,87099.0,2806,33
3,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,8.70,5,4.22,16831.0,103,34
4,291457.0,Gloomhaven: Jaws of the Lion,2020.0,1,4,120,14,8392,8.87,6,3.55,21609.0,796,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,1410.0,Trouble,1965.0,2,4,45,4,3255,3.79,20339,1.05,4962.0,5723,11
15253,16398.0,War,0.0,2,2,30,4,1340,2.28,20340,1.00,427.0,0,11
15254,5048.0,Candy Land,1949.0,2,4,30,3,4006,3.18,20342,1.08,5788.0,5723,11
15255,5432.0,Chutes and Ladders,-200.0,2,6,30,3,3783,2.86,20343,1.02,4400.0,4061,11


Из логики id и имя игры вообще никак не влияют на рейтинг игры, поэтому не буду использовать в регрессии

In [112]:
X = df0.drop(['ID', 'Name', 'Rating Average'], axis=1)
Y = df0['Rating Average']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3)

In [113]:
model1 = LinearRegression()
model1.fit(X_train, Y_train)

Y_pred = model1.predict(X_test)

# print('Веса всех признаков (x1, ..., x16): ', model.coef_)
# print('Свободный коэффицент: ', model.intercept_)

print('Mean Absolute Error: ', mean_absolute_error(Y_test, Y_pred))
print('Mean Squared Error: ', mean_squared_error(Y_test, Y_pred))
print('R2: ', r2_score(Y_test, Y_pred))
#Результат очень даже не плох, но у гет_дамис лучше


Mean Absolute Error:  0.4325356865872353
Mean Squared Error:  0.3508735903295452
R2:  0.6060046003876174


In [101]:
df1

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,...,Zone of Control,0,Abstract Games,Children's Games,Customizable Games,Family Games,Party Games,Strategy Games,Thematic Games,Wargames
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,8.79,1,...,0,0,0,0,0,0,0,1,1,0
1,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,8.66,3,...,0,0,0,0,0,0,0,1,0,0
2,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,8.43,4,...,0,0,0,0,0,0,0,1,0,0
3,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,8.70,5,...,0,0,0,0,0,0,0,1,1,0
4,291457.0,Gloomhaven: Jaws of the Lion,2020.0,1,4,120,14,8392,8.87,6,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,1410.0,Trouble,1965.0,2,4,45,4,3255,3.79,20339,...,0,0,0,1,0,0,0,0,0,0
15253,16398.0,War,0.0,2,2,30,4,1340,2.28,20340,...,0,0,0,1,0,0,0,0,0,0
15254,5048.0,Candy Land,1949.0,2,4,30,3,4006,3.18,20342,...,0,0,0,1,0,0,0,0,0,0
15255,5432.0,Chutes and Ladders,-200.0,2,6,30,3,3783,2.86,20343,...,0,0,0,1,0,0,0,0,0,0


Попробую масштабирование к df1

In [115]:
#нормализация:
mmsc = MinMaxScaler()
columns1=['Play Time','Users Rated','BGG Rank','Year Published','Users Rated']
df1[columns1] = mmsc.fit_transform(df1[columns1]) # type - numpy array

x = df1.drop(['ID', 'Name', 'Rating Average'], axis=1)
y = df1['Rating Average']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
model2 = LinearRegression()
model2.fit(x_train, y_train)

y_pred = model2.predict(x_test)

# print('Веса всех признаков (x1, ..., x16): ', model.coef_)
# print('Свободный коэффицент: ', model.intercept_)

print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
print('R2: ', r2_score(y_test, y_pred))
#еще лучше
a=x_train
b=x_test
c=y_train
d=y_test

Mean Absolute Error:  0.3968958037861129
Mean Squared Error:  0.29345656095093176
R2:  0.6680391340430898


стандартизация

In [116]:
std = StandardScaler()
df2[columns1] = std.fit_transform(df2[columns1]) # type - numpy array
x = df1.drop(['ID', 'Name', 'Rating Average'], axis=1)
y = df1['Rating Average']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
model3 = LinearRegression()
model3.fit(x_train, y_train)

y_pred = model3.predict(x_test)

# print('Веса всех признаков (x1, ..., x16): ', model.coef_)
# print('Свободный коэффицент: ', model.intercept_)

print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
print('R2: ', r2_score(y_test, y_pred))
#чуть хуже чем при нормализации

Mean Absolute Error:  0.3952101898358633
Mean Squared Error:  0.2860685175585604
R2:  0.6641737242255579


к нормализации еще knn сделаю


In [120]:
x = df1.drop(['ID', 'Name', 'Rating Average'], axis=1)
y = df1['Rating Average']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
from sklearn.neighbors import KNeighborsRegressor # регрессор

from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score

scores = []

for i in range(1,25):
    score = {}
    knn = KNeighborsRegressor(i)
    knn.fit(x_train, y_train)
    answers = knn.predict(x_test)

    score['k'] = i
    score['MAE'] = mean_absolute_error(answers, y_test)
    score['MSE'] = mean_squared_error(answers,y_test)
    score['R2'] = r2_score(answers,y_test)

    scores.append(score)

scores = pd.DataFrame(scores)
scores


,k,MAE,MSE,R2
0,1,0.838775,1.234605,-0.509058
1,2,0.740941,0.951753,-0.896008
2,3,0.710361,0.873531,-1.223593
3,4,0.695515,0.836538,-1.450027
4,5,0.686010,0.818835,-1.646008
5,6,0.679292,0.799170,-1.826146
6,7,0.670805,0.782025,-2.020390
7,8,0.665125,0.768941,-2.186751
8,9,0.662218,0.764313,-2.360584
9,10,0.661718,0.761790,-2.469586


In [121]:
i = 6

knn = KNeighborsRegressor(i)
knn.fit(x_train, y_train)
answers = knn.predict(x_test)

print('Mean Absolute Error:', mean_absolute_error(answers, y_test))
print('Mean Squared Error:', mean_squared_error(answers,y_test))
print('R2 score:', r2_score(answers,y_test))
#knn работает хуже

Mean Absolute Error: 0.679292267365662
Mean Squared Error: 0.7991700657735061
R2 score: -1.8261463864092953


In [122]:
from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(max_depth=4, random_state=1)

reg.fit(x_train, y_train)
y_pred_tree = reg.predict(x_test)

print('Mean Absolute Error:', mean_absolute_error(y_pred_tree, y_test))
print('Mean Squared Error:', mean_squared_error(y_pred_tree, y_test))
print('R2 score:', r2_score(y_pred_tree, y_test))
#с деревом решений дало лучший результат

Mean Absolute Error: 0.3638009296008538
Mean Squared Error: 0.2458047255772239
R2 score: 0.6235240855936439


теперь обработаю test_data

In [158]:
dff=pd.read_csv('test_data (1).csv')
print(dff)
dff['Domains'] = dff['Domains'].fillna('0')
dff['Owned Users'] = dff['Owned Users'].fillna(0)
dff['Mechanics'] = dff['Mechanics'].fillna('0')
dff['Year Published'] = dff['Year Published'].fillna(0)
dff['ID'] = dff['ID'].fillna('-1')

dff['Complexity Average']=dff['Complexity Average'].str.replace(',','.')
dff['Complexity Average']=dff['Complexity Average'].astype(float)
dfff = dff

df1_gt = dff['Mechanics'].str.get_dummies(sep=', ')
dff = pd.concat([dff.drop(columns=['Mechanics']), df1_gt], axis=1)

df1_gt = dff['Domains'].str.get_dummies(sep=', ')
dff = pd.concat([dff.drop(columns=['Domains']), df1_gt], axis=1)

dff.isna().sum()


            ID                             Name  Year Published  Min Players  \
0     161936.0        Pandemic Legacy: Season 1          2015.0            2   
1      12333.0                Twilight Struggle          2005.0            2   
2     115746.0  War of the Ring: Second Edition          2012.0            2   
3     169786.0                           Scythe          2016.0            1   
4      28720.0                Brass: Lancashire          2007.0            2   
...        ...                              ...             ...          ...   
5081    6932.0                  Hi Ho! Cherry-O          1960.0            2   
5082    3510.0              Battle of the Sexes          1997.0            2   
5083    5895.0             Hungry Hungry Hippos          1978.0            2   
5084    2679.0                       Mouse Trap          1963.0            2   
5085    7316.0                            Bingo          1530.0            2   

      Max Players  Play Time  Min Age  

ID                0
Name              0
Year Published    0
Min Players       0
Max Players       0
                 ..
Family Games      0
Party Games       0
Strategy Games    0
Thematic Games    0
Wargames          0
Length: 196, dtype: int64

In [159]:
mmsc = MinMaxScaler()
columns1=['Play Time','Users Rated','BGG Rank','Year Published','Users Rated']
dff[columns1] = mmsc.fit_transform(dff[columns1]) # type - numpy array

In [160]:
test = dff.drop(['ID', 'Name'], axis=1)

In [174]:
unique_columns_df1 = x.columns.difference(test.columns)
unique_columns_dff = test.columns.difference(x.columns)
print("Уникальные столбцы в df1:")
print(unique_columns_df1)
print("Уникальные столбцы в dff:")
print(unique_columns_dff)


Уникальные столбцы в df1:
Index([], dtype='object')
Уникальные столбцы в dff:
Index([], dtype='object')


In [168]:
new_columns = ['Auction: English', 'Auction: Once Around', 'Closed Economy Auction',
       'Different Dice Movement', 'Hot Potato', 'Multiple-Lot Auction',
       'Turn Order: Random']

for column in new_columns:
    test[column] = 0


KeyError: "['ID' 'Name' 'Rating Average'] not found in axis"

In [163]:
unique_columns_df1 = x.columns.difference(test.columns)
unique_columns_dff = test.columns.difference(x.columns)
print("Уникальные столбцы в df1:")
print(unique_columns_df1)
print("Уникальные столбцы в dff:")
print(unique_columns_dff)

Уникальные столбцы в df1:
Index([], dtype='object')
Уникальные столбцы в dff:
Index([], dtype='object')


In [197]:
test

,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,BGG Rank,Complexity Average,Owned Users,0,...,Zone of Control,0,Abstract Games,Children's Games,Customizable Games,Family Games,Party Games,Strategy Games,Thematic Games,Wargames
0,0.998805,2,4,0.004167,13,0.408680,0.000000,2.84,65294.0,0,...,0,0,0,0,0,0,0,1,1,0
1,0.996813,2,2,0.012500,13,0.400538,0.000393,3.59,56219.0,0,...,0,0,0,0,0,0,0,1,0,1
2,0.998208,2,4,0.012500,13,0.134498,0.000492,4.14,22281.0,0,...,0,0,0,0,0,0,0,0,1,1
3,0.999004,1,5,0.007986,14,0.568054,0.000590,3.41,75640.0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.997212,2,4,0.008333,14,0.190232,0.000836,3.86,25429.0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5081,0.987851,2,4,0.000694,3,0.009870,0.999213,1.03,1691.0,0,...,0,0,0,1,0,0,0,0,0,0
5082,0.995220,2,8,0.003125,12,0.010410,0.999361,1.08,1987.0,0,...,0,0,0,0,0,0,1,0,0,0
5083,0.991436,2,4,0.000694,4,0.022893,0.999459,1.05,2568.0,1,...,0,1,0,1,0,0,0,0,0,0
5084,0.988449,2,4,0.002083,6,0.026153,0.999607,1.11,3408.0,0,...,0,0,0,1,0,0,0,0,0,0


In [196]:
x

,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,BGG Rank,Complexity Average,Owned Users,0,...,Zone of Control,0,Abstract Games,Children's Games,Customizable Games,Family Games,Party Games,Strategy Games,Thematic Games,Wargames
0,0.999095,1,4,0.002000,14,0.411268,0.000000,3.86,68323.0,0,...,0,0,0,0,0,0,0,1,1,0
1,0.999276,2,4,0.002000,14,0.187769,0.000098,3.91,28785.0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.998913,1,5,0.002000,12,0.634483,0.000147,3.24,87099.0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.999095,3,6,0.008000,14,0.131508,0.000197,4.22,16831.0,0,...,0,0,0,0,0,0,0,1,1,0
4,0.999638,1,4,0.002000,14,0.081833,0.000246,3.55,21609.0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15252,0.989678,2,4,0.000750,4,0.031561,0.999754,1.05,4962.0,0,...,0,0,0,1,0,0,0,0,0,0
15253,0.633828,2,2,0.000500,4,0.012820,0.999803,1.00,427.0,1,...,0,0,0,1,0,0,0,0,0,0
15254,0.986780,2,4,0.000500,3,0.038910,0.999902,1.08,5788.0,0,...,0,0,0,1,0,0,0,0,0,0
15255,0.597610,2,6,0.000500,3,0.036728,0.999951,1.02,4400.0,0,...,0,0,0,1,0,0,0,0,0,0


In [193]:
new_columns_order = x.columns.tolist()
test = test.loc[:, ~test.columns.duplicated()]
test = test[new_columns_order]

In [198]:
ans = reg.predict(test)

my_submition = pd.DataFrame(ans, columns=['Rating Average'])

my_submition.reset_index().to_csv('answerss.csv', index=False)